In [1]:
###############################################################
# 1) load packages
###############################################################
library(markovMSM)
library(mstate)
library(survival)
library(dplyr)

###############################################################
# 2) define a function that builds and returns Q as a 2D array
###############################################################
compute_Q_array <- function() {
  # 2a) load ebmt4 and build tmat
  data("ebmt4")
  db_wide <- ebmt4

  positions <- list(
    c(2, 3, 5, 6),
    c(4, 5, 6),
    c(4, 5, 6),
    c(5, 6),
    c(6),
    c()
  )
  state.names <- c("Tx", "Rec", "AE", "Rec+AE", "Rel", "Death")
  tmat <- transMatMSM(positions, state.names)

  timesNames <- c(NA, "rec", "ae", "recae", "rel", "srv")
  status    <- c(NA, "rec.s", "ae.s", "recae.s", "rel.s", "srv.s")

  # 2b) convert to “long” format, then drop invalid intervals
  db_long <- prepMSM(
    data       = db_wide,
    trans      = tmat,
    timesNames = timesNames,
    status     = status
  )
  db_long <- subset(db_long, Tstop > Tstart)
  db_long$trans <- as.factor(db_long$trans)

  # 2c) fit a stratified Cox model
  cox_fit <- coxph(
    Surv(Tstart, Tstop, status) ~ strata(trans),
    data = db_long
  )

  # 2d) extract baseline cumulative hazards
  bh_all   <- basehaz(cox_fit, centered = FALSE)
  bh_split <- split(bh_all, bh_all$strata)

  # 2e) build an empty 6×6 matrix of zeros
  Qmat <- matrix(0, nrow = 6, ncol = 6)
  rownames(Qmat) <- colnames(Qmat) <- state.names

  for (j in seq_along(bh_split)) {
    df_j     <- bh_split[[j]]
    last_row <- df_j[nrow(df_j), ]
    t_max    <- last_row$time
    H_j      <- last_row$hazard
    λ_j      <- H_j / t_max   # constant‐rate approximation

    idx      <- which(tmat == j, arr.ind = TRUE)
    i        <- idx[1]
    k        <- idx[2]
    Qmat[i, k] <- λ_j
  }

  for (i in 1:6) {
    Qmat[i, i] <- -sum(Qmat[i, -i])
  }

  # 2f) explicitly coerce to a 2D array (identical to a matrix),
  #    but now we “return” it.
  Q_array <- array(Qmat,
                   dim      = dim(Qmat),
                   dimnames = list(state.names, state.names))
  return(Q_array)
}

###############################################################
# 3) call the function and store the result
###############################################################
Q_as_2d_array <- compute_Q_array()

# 4) inspect:
str(Q_as_2d_array)
#>  num [1:6, 1:6] -3.32e-04 0 0 0 0 0 1.25e-04 -2.23e-04 0 0 0 0 1.07e-04 0 -2.70e-04 0 0 0 ...
#>  - attr(*, "dimnames")=List of 2
#>   ..$ : chr [1:6] "Tx" "Rec" "AE" "Rec+AE" ...
#>   ..$ : chr [1:6] "Tx" "Rec" "AE" "Rec+AE" ...

# 5) print to confirm
Q_as_2d_array



Loading required package: survival


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




 num [1:6, 1:6] -0.000332 0 0 0 0 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:6] "Tx" "Rec" "AE" "Rec+AE" ...
  ..$ : chr [1:6] "Tx" "Rec" "AE" "Rec+AE" ...


,Tx,Rec,AE,Rec+AE,Rel,Death
Tx,-0.000331941,0.0001246943,0.0001072461,0.0000000000,4.065842e-05,5.934219e-05
Rec,0.000000000,-0.0002228618,0.0000000000,0.0001586238,4.551207e-05,1.872588e-05
AE,0.000000000,0.0000000000,-0.0002703439,0.0001454130,3.169115e-05,9.323981e-05
Rec+AE,0.000000000,0.0000000000,0.0000000000,-0.0001128961,4.065411e-05,7.224203e-05
Rel,0.000000000,0.0000000000,0.0000000000,0.0000000000,-1.242991e-03,1.242991e-03
Death,0.000000000,0.0000000000,0.0000000000,0.0000000000,0.000000e+00,0.000000e+00
